In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))


In [2]:
import yfinance as yf
from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime

import random
random.seed(42)

In [7]:
df = yf.Ticker("MSFT").history(period='max')
df


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1986-03-13 00:00:00-05:00,0.054594,0.062623,0.054594,0.059946,1031788800,0.0,0.0
1986-03-14 00:00:00-05:00,0.059947,0.063158,0.059947,0.062087,308160000,0.0,0.0
1986-03-17 00:00:00-05:00,0.062087,0.063694,0.062087,0.063158,133171200,0.0,0.0
1986-03-18 00:00:00-05:00,0.063158,0.063694,0.061017,0.061552,67766400,0.0,0.0
1986-03-19 00:00:00-05:00,0.061552,0.062087,0.059946,0.060482,47894400,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-17 00:00:00-04:00,440.230011,441.850006,432.269989,435.149994,18874200,0.0,0.0
2024-09-18 00:00:00-04:00,435.000000,436.029999,430.410004,430.809998,18898000,0.0,0.0
2024-09-19 00:00:00-04:00,441.230011,441.500000,436.899994,438.690002,21706600,0.0,0.0


In [23]:
from backtesting import Strategy
import talib as ta

class MeanReversion(Strategy):
    # Parámetros ajustables de la estrategia
    lookback_period = 50   # Periodo de la SMA
    deviation_threshold = 0.01  # 5% de desviación para generar señal
    risk = 1  # Tamaño del riesgo (posición)
    
    def init(self):
        # Calcular la media móvil simple (SMA)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.lookback_period)
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)

    def next(self):
        # Precio actual y valor de la SMA
        actual_close = self.data.Close[-1]
        sma_value = self.sma[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]

        # Desviación del precio con respecto a la SMA (en porcentaje)
        deviation = (actual_close - sma_value) / sma_value

        if self.position:
            if self.position.is_long and actual_close >= self.sma:
                self.position.close()

            if self.position.is_short and actual_close <= self.sma:
                self.position.close()  
        
        else:
            # Condiciones para comprar (precio por debajo de la SMA más del umbral de desviación)
            if deviation <= -self.deviation_threshold and cum_rsi <= 25:
                self.buy(size=self.risk / 100)

            # Condiciones para vender (precio por encima de la SMA más del umbral de desviación)
            elif deviation >= self.deviation_threshold and cum_rsi >= 75:
                self.sell(size=self.risk / 100)
    
    def next_live(self, positions):
        # Precio actual y valor de la SMA
        actual_close = self.data.Close[-1]
        actual_date = self.data.index[-1]
        sma_value = self.sma[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]

        # Desviación del precio con respecto a la SMA (en porcentaje)
        deviation = (actual_close - sma_value) / sma_value

        if positions:
            if positions[0] == mt5.POSITION_TYPE_BUY and actual_close >= self.sma:
                print(actual_date, 'cerrar largo')

            if positions[0] == mt5.POSITION_TYPE_SELL and actual_close <= self.sma:
                print(actual_date, 'cerrar corto')
  
        
        else:
            # Condiciones para comprar (precio por debajo de la SMA más del umbral de desviación)
            if deviation <= -self.deviation_threshold and cum_rsi <= 25:
                print(actual_date, 'abrir_largo')

            # Condiciones para vender (precio por encima de la SMA más del umbral de desviación)
            elif deviation >= self.deviation_threshold and cum_rsi >= 75:
                print(actual_date, 'abrir corto')
    


In [24]:
bt_train = Backtest(
    df, 
    MeanReversion,
    commission=7e-4,
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()
stats

Start                     1986-03-13 00:00...
End                       2024-09-23 00:00...
Duration                  14073 days 23:00:00
Exposure Time [%]                   90.360453
Equity Final [$]                   2363.13424
Equity Peak [$]                  16805.553637
Return [%]                         -84.245772
Buy & Hold Return [%]           723061.684544
Return (Ann.) [%]                   -4.683004
Volatility (Ann.) [%]               21.121391
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -92.012403
Avg. Drawdown [%]                   -9.479921
Max. Drawdown Duration    13867 days 00:00:00
Avg. Drawdown Duration     1270 days 00:00:00
# Trades                                  359
Win Rate [%]                         72.70195
Best Trade [%]                      15.145664
Worst Trade [%]                    -238.38885
Avg. Trade [%]                    

In [25]:
bt_train._results._strategy.next_live(positions=[mt5.ORDER_TYPE_BUY])

2024-09-23 00:00:00-04:00 cerrar largo
